In [2]:
import random
import numpy as np

class service():
  def __init__(self):
    self.count = 0
    self.nome = []
    self.chega = []
    self.inicioAtd = []
    self.fimAtd = []

tec_valor_arr = []
tec_observe_arr = []
ts_valor_arr = []
ts_observe_arr = []

def mmc(valor_arr, observe_arr):
  random_number = random.uniform(0, 1)
  for i in range(len(observe_arr)):
    if random_number < observe_arr[i]:
      return valor_arr[i]


def convertToPercentage(observe_arr):
  total_observe = sum(observe_arr)
  local_observe = 0
  for i in range(len(observe_arr)):
    local_observe += observe_arr[i]
    observe_arr[i] = local_observe / total_observe


def createMCCTable(tec_type, ts_type):
  if tec_type != 'd' and tec_type != 'D':
    print('\n\n\n------------------Distribuição de Frequências dos TEC--------------\n\n')
    n_class = int(input("Qual é o numero de classes para TEC: "))

    for i in range(n_class):
      print(f'\n{i+1}º Classe')
      class_valor = int(input("Valor da classe: "))
      tec_valor_arr.append(class_valor)
      n_observe = int(input("Número de observações: "))
      tec_observe_arr.append(n_observe)
    
    convertToPercentage(tec_observe_arr)

  if ts_type != 'd' and ts_type != 'D':
    print('\n\n\n------------------Distribuição de Frequências dos TS---------------\n\n')
    n_class = int(input("Qual é o numero de classes para TS: "))

    for i in range(n_class):
      print(f'\n{i+1}º Classe')
      class_valor = int(input("Valor da classe: "))
      ts_valor_arr.append(class_valor)
      n_observe = int(input("Número de observações: "))
      ts_observe_arr.append(n_observe)
    
    convertToPercentage(ts_observe_arr)


def fixArrays(array1, arra2):
  while(True):
    if len(array1) < len(arra2):
      arra2.pop()
    else:
      return


def printStats(s, tempoTotal):
  fixArrays(s.inicioAtd, s.chega)

  npTempoChegadas = np.array(s.chega)
  npTempoInicioAtd = np.array(s.inicioAtd)
  npTempoTerminoAtd = np.array(s.fimAtd)

  tempoTotalEsperaFila = np.sum(npTempoInicioAtd - npTempoChegadas)
  print('Tempo total de espera na fila =', tempoTotalEsperaFila)

  numeroTotalCliente = len(npTempoChegadas)
  print('Número total de clientes =', numeroTotalCliente)

  numeroClienteEsperaram = 0
  for i in range(numeroTotalCliente):
    if npTempoChegadas[i] != npTempoInicioAtd[i]:
      numeroClienteEsperaram += 1
  print('Numero de clientes que esperaram = ', numeroClienteEsperaram)

  tempoLivreOperador = npTempoInicioAtd[0]
  for i in range(numeroTotalCliente - 1):
    tempoLivreOperador += npTempoInicioAtd[i+1] - npTempoTerminoAtd[i] 
  print('Tempo livre do operadordor =', tempoLivreOperador)

  tempoTotalServico = np.sum(npTempoTerminoAtd - npTempoInicioAtd)
  print('Tempo total de serviço =', tempoTotalServico)

  tempoTotalSistema = tempoTotalServico + tempoTotalEsperaFila
  print('Tempo total no sistema = ', tempoTotalSistema)

  tempoMedioEsperaFila = tempoTotalEsperaFila/numeroTotalCliente
  print('Tempo médio de espera da fila = ', tempoMedioEsperaFila)

  probClienteEsperaFila = numeroClienteEsperaram/numeroTotalCliente
  print('Probabilidade de um ciente esperar na fila = ', probClienteEsperaFila)

  probOperadorLivre = tempoLivreOperador/tempoTotal
  print('Probabilidade do operador livre = ', probOperadorLivre)

  tempoMedioServico = tempoTotalServico/numeroTotalCliente
  print('Tempo médio de serviço = ', tempoMedioServico)

  tempoMedioSistema = tempoTotalSistema/numeroTotalCliente
  print('Tempo médio despendido no sistemas = ', tempoMedioSistema)


def chegada(time, state, n_fila, hc, hs, s, fila_fixa, tec_type, ts_type):
  time = hc

  if state == 0: # chega e inicia o serviço
    state = 1
    if ts_type == 'd' or ts_type == 'D':
      ts = int(input("Qual o TS: "))
    else:
      ts = int(mmc(ts_valor_arr, ts_observe_arr))
    hs = time + ts
    
    s.count += 1 
    s.nome.append(s.count)
    s.chega.append(hc)
    s.inicioAtd.append(time)
    s.fimAtd.append(hs)

  else: # chega mas n inicia o serviço
    if n_fila < fila_fixa:
      s.count += 1 
      s.nome.append(s.count)
      s.chega.append(hc)
      n_fila = n_fila + 1
    else:
      print('FILA LOTADA!!!')
    
  # proximo evento chegada ocorrera no tempo time+tec
  if tec_type == 'd' or tec_type == 'D':
    tec = int(input("Qual a TEC: "))
  else:
    tec = int(mmc(tec_valor_arr, tec_observe_arr))
  hc = time + tec

  return time, state, n_fila, hc, hs


def saida(time, state, n_fila, hc, hs, s, ts_type):
  time = hs

  if n_fila>0: # finaliza o serviço atual e inicia o proximo
    n_fila = n_fila - 1
    if ts_type == 'd' or ts_type == 'D':
      ts = int(input("Qual o TS: "))
    else:
      ts = int(mmc(ts_valor_arr, ts_observe_arr))
    hs = time + ts

    s.inicioAtd.append(time)
    s.fimAtd.append(hs)

  else: # finaliza o atual
    state = 0
    hs = 9999

  return time, state, n_fila, hc, hs


def simu(fila_fixa, tec_type, ts_type, s, n_simu):
  time, state, n_fila, hc, hs = 0, 0, 0, 0, 9999
  for _ in range(n_simu):

    # qual o proximo evento? de chegada ou saida?
    if hc<hs:
      time, state, n_fila, hc, hs = chegada(time, state, n_fila, hc, hs, 
                                            s, fila_fixa, tec_type, ts_type)
      print(f"Evento Chegada | Cliente {s.nome[-1]} | TR {time} | " + \
            f"ES {state} | TF {n_fila} | HC {hc} | HS {hs}")
      
    else:
      time, state, n_fila, hc, hs = saida(time, state, n_fila, 
                                          hc, hs, s, ts_type)
      print(f"Evento Saída | Cliente {s.nome[s.count-n_fila-2]} | TR {time} " + \
            f"| ES {state} | TF {n_fila} | HC {hc} | HS {hs}")
  return time


def main():
  print('\n-------------------------CONFIGURAR SIMULAÇÃO-------------------------\n\n')
  
  fila = input("A fila terá tamanho fixo? (s/n) ")

  if fila == 's' or fila == 'S':
    fila_fixa = int(input('Qual o tamanho: '))
  else:
    fila_fixa = np.inf

  tec_type = input("O Tempo entre Chegada será deterministico (d) ou aleatorio (a): ")
  ts_type = input("O Tempo de Serviço será deterministico (d) ou aleatorio (a): ")
  n_simu = int(input("Quantos eventos você quer que simule: "))
  s = service()
    
  createMCCTable(tec_type, ts_type)
  print('\n\n\n-------------------------INICIANDO SIMULAÇÃO-------------------------\n\n')
  time_total = simu(fila_fixa, tec_type, ts_type, s, n_simu)
  
  print('\n\n\n-------------------------RELATORIO FINAL-------------------------\n\n')
  printStats(s, time_total)
main()


-------------------------CONFIGURAR SIMULAÇÃO-------------------------


A fila terá tamanho fixo? (s/n) n
O Tempo entre Chegada será deterministico (d) ou aleatorio (a): a
O Tempo de Serviço será deterministico (d) ou aleatorio (a): a
Quantos eventos você quer que simule: 10



------------------Distribuição de Frequências dos TEC--------------


Qual é o numero de classes para TEC: 3

1º Classe
Valor da classe: 1
Número de observações: 5

2º Classe
Valor da classe: 2
Número de observações: 7

3º Classe
Valor da classe: 3
Número de observações: 10



------------------Distribuição de Frequências dos TS---------------


Qual é o numero de classes para TS: 4

1º Classe
Valor da classe: 2
Número de observações: 4

2º Classe
Valor da classe: 5
Número de observações: 4

3º Classe
Valor da classe: 2
Número de observações: 4

4º Classe
Valor da classe: 6
Número de observações: 4



-------------------------INICIANDO SIMULAÇÃO-------------------------


Evento Chegada | Cliente 1 | TR 0 | ES